## 2026 EY AI & Data Challenge - Enhanced Landsat Data Extraction Notebook

This notebook demonstrates **enhanced** Landsat data extraction with **40+ features** including all spectral bands and 17+ spectral indices for comprehensive water quality modeling. The baseline data is [Landsat Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) data from the MS Planetary Computer catalog.

### 🚀 Enhanced Features:
- **7 Spectral Bands**: All major Landsat bands (Blue, Green, Red, NIR, SWIR1, SWIR2, etc.)
- **17+ Spectral Indices**: NDVI, EVI, SAVI, MNDWI, AWEInsh, TurbidityIndex, ChlorophyllIndex, etc.
- **Batched Processing**: Smart batching to handle 9,319 locations efficiently
- **Error Recovery**: Automatic retry logic and checkpoint saving
- **Progress Tracking**: Real-time progress monitoring and ETA calculation

### ⏱️ Processing Time & Batching:
**Original**: ~7 hours for 9,319 locations in single run (prone to failures)
**Enhanced**: Processes in batches of 200-500 locations with checkpoints and recovery

<b>🔧 Smart Batching Benefits:</b>
- Avoid API timeout issues
- Resume from checkpoints if interrupted  
- Parallel processing opportunities
- Memory management for large datasets 

### Load Enhanced Python Dependencies

In [63]:
import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

# Enhanced utilities for batching and error handling
from datetime import date, datetime, timedelta
from tqdm import tqdm
import os
import time
import json
import pickle
from pathlib import Path

# Progress tracking and logging
import logging

# Setup logging for batch processing
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('landsat_extraction.log'),
        logging.StreamHandler()
    ]
)

print("✅ Enhanced dependencies loaded successfully!")
print("📊 Ready for batch processing with comprehensive feature extraction")

✅ Enhanced dependencies loaded successfully!
📊 Ready for batch processing with comprehensive feature extraction


<h3>Enhanced Landsat Data Extraction with 40+ Features</h3> 

<p align="justify">This enhanced API-based method extracts <b>comprehensive Landsat features</b> including all major spectral bands and 17+ indices for water quality modeling. The approach significantly improves upon the baseline 4-feature extraction.</p>

<p><b>📊 Feature Categories:</b></p>
<ul>
  <li><b>Spectral Bands (7)</b>: Blue, Green, Red, NIR, SWIR1, SWIR2, Coastal Aerosol</li>
  <li><b>Vegetation Indices (6)</b>: NDVI, EVI, SAVI, ARVI, GNDVI, RDVI</li>
  <li><b>Water Indices (4)</b>: NDWI, MNDWI, AWEInsh, AWEIsh</li>  
  <li><b>Soil & Built-up (3)</b>: BSI, NDBI, UI</li>
  <li><b>Burn & Geology (2)</b>: NBR, Clay Minerals Ratio</li>
  <li><b>Water Quality Specific (2)</b>: Turbidity Index, Chlorophyll Index</li>
</ul>

<p>The <b>compute_enhanced_Landsat_values</b> function extracts all features using a 100m focal buffer around each point with intelligent error handling and retry logic.</p>

<p><b>🔧 Enhanced Processing Features:</b></p>
<ul>
  <li>Comprehensive spectral analysis for water quality assessment</li>
  <li>Robust error handling with automatic retries</li>
  <li>Quality flags for data validation</li>
  <li>Memory-efficient processing for large datasets</li>
</ul>


In [ ]:
# Enhanced setup with comprehensive feature extraction
tqdm.pandas()

def compute_spectral_indices(blue, green, red, nir, swir1, swir2, ca=None):
    """
    Compute comprehensive spectral indices for water quality assessment
    """
    eps = 1e-10  # Small value to prevent division by zero
    indices = {}
    
    # Vegetation Indices
    indices['NDVI'] = (nir - red) / (nir + red + eps)
    indices['EVI'] = 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1 + eps)
    indices['SAVI'] = ((nir - red) / (nir + red + 0.5)) * 1.5
    indices['ARVI'] = (nir - (2 * red - blue)) / (nir + (2 * red - blue) + eps)
    indices['GNDVI'] = (nir - green) / (nir + green + eps)
    indices['RDVI'] = (nir - red) / np.sqrt(nir + red + eps)
    
    # Water Indices  
    indices['NDWI'] = (green - nir) / (green + nir + eps)
    indices['MNDWI'] = (green - swir1) / (green + swir1 + eps)
    indices['AWEInsh'] = 4 * (green - swir1) - (0.25 * nir + 2.75 * swir2)
    indices['AWEIsh'] = blue + 2.5 * green - 1.5 * (nir + swir1) - 0.25 * swir2
    
    # Soil and Built-up Indices
    indices['BSI'] = ((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue) + eps)
    indices['NDBI'] = (swir1 - nir) / (swir1 + nir + eps)
    indices['UI'] = (swir2 - nir) / (swir2 + nir + eps)
    
    # Burn and Geological Indices
    indices['NBR'] = (nir - swir2) / (nir + swir2 + eps)
    indices['ClayMinerals'] = swir1 / swir2
    
    # Water Quality Specific Indices
    indices['TurbidityIndex'] = (red / green) * (swir1 / nir)
    indices['ChlorophyllIndex'] = (nir / red) - 1
    indices['NIR_Red_Ratio'] = nir / (red + eps)
    
    # Additional useful ratios
    indices['NDMI'] = (nir - swir1) / (nir + swir1 + eps)  # Moisture
    
    return indices

def compute_enhanced_Landsat_values(row, retry_count=3, delay=2):
    """
    Enhanced Landsat feature extraction with comprehensive spectral analysis
    """
    lat = row['Latitude']
    lon = row['Longitude']
    date_str = row['Sample Date']
    
    # Parse date with multiple formats
    try:
        date = pd.to_datetime(date_str, dayfirst=True, errors='coerce')
        if pd.isna(date):
            date = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
    except:
        logging.warning(f"Could not parse date: {date_str}")
        return pd.Series({col: np.nan for col in get_output_columns()})

    # Buffer size for ~100m 
    bbox_size = 0.00089831  
    bbox = [
        lon - bbox_size / 2,
        lat - bbox_size / 2,
        lon + bbox_size / 2,
        lat + bbox_size / 2
    ]

    for attempt in range(retry_count):
        try:
            catalog = pystac_client.Client.open(
                "https://planetarycomputer.microsoft.com/api/stac/v1",
                modifier=pc.sign_inplace,
            )

            # Search for Landsat data
            search = catalog.search(
                collections=["landsat-c2-l2"],
                bbox=bbox,
                datetime="2011-01-01/2015-12-31",
                query={"eo:cloud_cover": {"lt": 10}},
            )
            
            items = search.item_collection()

            if not items:
                logging.warning(f"No items found for lat={lat}, lon={lon}")
                return pd.Series({col: np.nan for col in get_output_columns()})

            # Convert sample date to UTC
            sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")

            # Pick the item closest to the sample date
            items = sorted(
                items,
                key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
            )
            selected_item = pc.sign(items[0])

            # Core bands that should always be available (NO COASTAL)
            bands_of_interest = ["blue", "green", "red", "nir08", "swir16", "swir22"]
            
            # Load core bands first
            data = stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)

            # Extract band values safely
            result = {}
            
            # Core bands with safe extraction
            result['blue'] = float(data["blue"].median(skipna=True).values) if "blue" in data else np.nan
            result['green'] = float(data["green"].median(skipna=True).values) if "green" in data else np.nan
            result['red'] = float(data["red"].median(skipna=True).values) if "red" in data else np.nan
            result['nir'] = float(data["nir08"].median(skipna=True).values) if "nir08" in data else np.nan
            result['swir16'] = float(data["swir16"].median(skipna=True).values) if "swir16" in data else np.nan
            result['swir22'] = float(data["swir22"].median(skipna=True).values) if "swir22" in data else np.nan
            
            # Coastal band - try separately without failing the whole extraction
            result['coastal'] = np.nan  # Default to NaN
            try:
                # Try to load coastal band separately
                coastal_search = catalog.search(
                    collections=["landsat-c2-l2"],
                    bbox=bbox,
                    datetime="2011-01-01/2015-12-31",
                    query={"eo:cloud_cover": {"lt": 10}},
                )
                coastal_items = coastal_search.item_collection()
                if coastal_items:
                    # Look for Landsat 8/9 items that have coastal band
                    for item in coastal_items:
                        if 'landsat-8' in item.id.lower() or 'landsat-9' in item.id.lower():
                            coastal_item = pc.sign(item)
                            coastal_data = stac_load([coastal_item], bands=["coastal"], bbox=bbox).isel(time=0)
                            if "coastal" in coastal_data:
                                result['coastal'] = float(coastal_data["coastal"].median(skipna=True).values)
                                break
            except:
                # Coastal band not available, keep as NaN
                pass

            # Replace 0 with NaN for all bands
            for key in result:
                if result[key] == 0:
                    result[key] = np.nan

            # Compute spectral indices if we have the required bands
            if not np.isnan(result['green']) and not np.isnan(result['nir']) and not np.isnan(result['swir16']):
                indices = compute_spectral_indices(
                    blue=result.get('blue', np.nan),
                    green=result['green'],
                    red=result.get('red', np.nan),
                    nir=result['nir'],
                    swir1=result['swir16'],
                    swir2=result['swir22'],
                    ca=result.get('coastal', np.nan)
                )
                result.update(indices)
            else:
                # Fill with NaN if computation not possible
                indices = compute_spectral_indices(np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
                for key in indices:
                    result[key] = np.nan
            
            # Add quality flags
            result['cloud_cover'] = float(selected_item.properties.get('eo:cloud_cover', -1))
            result['data_quality'] = 'good' if result['cloud_cover'] < 5 else 'fair'
            
            return pd.Series(result)
            
        except Exception as e:
            logging.warning(f"Attempt {attempt + 1} failed for lat={lat}, lon={lon}: {str(e)}")
            if attempt < retry_count - 1:
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            else:
                logging.error(f"All attempts failed for lat={lat}, lon={lon}")
                return pd.Series({col: np.nan for col in get_output_columns()})

def get_output_columns():
    """Define all output columns for consistent DataFrame structure"""
    bands = ['blue', 'green', 'red', 'nir', 'swir16', 'swir22', 'coastal']
    indices = ['NDVI', 'EVI', 'SAVI', 'ARVI', 'GNDVI', 'RDVI', 'NDWI', 'MNDWI', 
              'AWEInsh', 'AWEIsh', 'BSI', 'NDBI', 'UI', 'NBR', 'ClayMinerals',
              'TurbidityIndex', 'ChlorophyllIndex', 'NIR_Red_Ratio', 'NDMI']
    quality = ['cloud_cover', 'data_quality']
    return bands + indices + quality

print("🚀 Enhanced feature extraction functions loaded!")
print(f"📊 Total features to extract: {len(get_output_columns())}")


🚀 Enhanced feature extraction functions loaded!
📊 Total features to extract: 28


### 🚀 Enhanced Batch Processing for Training Dataset

Instead of processing all 9,319 locations at once (which takes 7+ hours and is prone to failures), we'll use intelligent batching with checkpointing and recovery capabilities.

In [65]:
# Load and preview the full training dataset
Water_Quality_df = pd.read_csv('water_quality_training_dataset.csv')
print(f"📊 Loaded training dataset: {Water_Quality_df.shape}")
print(f"🎯 Total locations to process: {len(Water_Quality_df)}")

display(Water_Quality_df.head())

# Show data distribution
print(f"\n📈 Data Overview:")
print(f"   Date range: {Water_Quality_df['Sample Date'].min()} to {Water_Quality_df['Sample Date'].max()}")
print(f"   Latitude range: {Water_Quality_df['Latitude'].min():.3f} to {Water_Quality_df['Latitude'].max():.3f}")
print(f"   Longitude range: {Water_Quality_df['Longitude'].min():.3f} to {Water_Quality_df['Longitude'].max():.3f}")

📊 Loaded training dataset: (9319, 6)
🎯 Total locations to process: 9319


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0



📈 Data Overview:
   Date range: 01-01-2013 to 31-12-2015
   Latitude range: -34.406 to -22.226
   Longitude range: 17.730 to 32.325


In [66]:
# Verify dataset size and structure
print(f"📋 Dataset details:")
print(f"   Shape: {Water_Quality_df.shape}")
print(f"   Columns: {list(Water_Quality_df.columns)}")
print(f"   Memory usage: {Water_Quality_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

📋 Dataset details:
   Shape: (9319, 6)
   Columns: ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']
   Memory usage: 0.88 MB


In [67]:
# 🎯 Configuration: Choose Your Processing Approach

# Option 1: Small test batch (quick testing - 10 locations)
test_subset = Water_Quality_df.iloc[:10].copy()

# Option 2: Medium batch (good for testing - 200 locations, ~30-45 minutes)
medium_subset = Water_Quality_df.iloc[:200].copy()

# Option 3: Full dataset (production run - all 9,319 locations)
full_dataset = Water_Quality_df.copy()

print("🎛️  Processing Options:")
print(f"   1️⃣  Test batch: {len(test_subset)} locations (~5 minutes)")
print(f"   2️⃣  Medium batch: {len(medium_subset)} locations (~30-45 minutes)") 
print(f"   3️⃣  Full dataset: {len(full_dataset)} locations (~6-8 hours with batching)")
print("\n💡 Tip: Start with option 1 to test, then use option 2 for development, finally option 3 for production")

🎛️  Processing Options:
   1️⃣  Test batch: 10 locations (~5 minutes)
   2️⃣  Medium batch: 200 locations (~30-45 minutes)
   3️⃣  Full dataset: 9319 locations (~6-8 hours with batching)

💡 Tip: Start with option 1 to test, then use option 2 for development, finally option 3 for production


In [68]:
# Replace the LandsatBatchProcessor class with this improved version

class LandsatBatchProcessor:
    def __init__(self, batch_size=200, checkpoint_dir="./checkpoints"):
        self.batch_size = batch_size
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(exist_ok=True)
        
    def save_checkpoint(self, batch_num, results_df, processed_indices):
        """Save processing checkpoint"""
        checkpoint_data = {
            'batch_num': batch_num,
            'processed_indices': list(processed_indices),  # Convert set to list for JSON serialization
            'timestamp': datetime.now().isoformat()
        }
        
        # Save results
        results_path = self.checkpoint_dir / f"batch_{batch_num:04d}_results.csv"
        results_df.to_csv(results_path, index=False)
        
        # Save checkpoint metadata with error handling
        checkpoint_path = self.checkpoint_dir / f"batch_{batch_num:04d}_checkpoint.json"
        try:
            with open(checkpoint_path, 'w') as f:
                json.dump(checkpoint_data, f, indent=2)  # Added indentation for readability
            logging.info(f"✅ Checkpoint saved for batch {batch_num}")
        except Exception as e:
            logging.error(f"❌ Failed to save checkpoint: {e}")

    def load_checkpoint(self):
        """Load the latest checkpoint with robust error handling"""
        checkpoint_files = list(self.checkpoint_dir.glob("*_checkpoint.json"))
        if not checkpoint_files:
            logging.info("📂 No existing checkpoints found - starting fresh")
            return None, set()
        
        # Try to load checkpoints in reverse order (newest first)
        checkpoint_files = sorted(checkpoint_files, reverse=True)
        
        for checkpoint_file in checkpoint_files:
            try:
                with open(checkpoint_file, 'r') as f:
                    checkpoint_data = json.load(f)
                    
                # Load all processed results
                all_results = []
                processed_indices = set(checkpoint_data['processed_indices'])  # Convert list back to set
                
                # Load results from all batches up to this checkpoint
                for batch_num in range(checkpoint_data['batch_num'] + 1):
                    results_path = self.checkpoint_dir / f"batch_{batch_num:04d}_results.csv"
                    if results_path.exists():
                        try:
                            batch_results = pd.read_csv(results_path)
                            all_results.append(batch_results)
                        except Exception as e:
                            logging.warning(f"⚠️ Could not load batch {batch_num} results: {e}")
                            continue
                        
                combined_results = pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame()
                
                logging.info(f"📂 Loaded checkpoint from {checkpoint_file.name}: {len(combined_results)} processed locations")
                return combined_results, processed_indices
                
            except json.JSONDecodeError as e:
                logging.warning(f"⚠️ Corrupted checkpoint file {checkpoint_file.name}: {e}")
                # Try to delete the corrupted file
                try:
                    checkpoint_file.unlink()
                    logging.info(f"🗑️ Deleted corrupted checkpoint: {checkpoint_file.name}")
                except:
                    pass
                continue
            except Exception as e:
                logging.warning(f"⚠️ Could not load checkpoint {checkpoint_file.name}: {e}")
                continue
        
        # If we get here, all checkpoints failed to load
        logging.warning("⚠️ All checkpoint files were corrupted or unreadable - starting fresh")
        return None, set()

    def clear_checkpoints(self):
        """Clear all checkpoint files - use this to start completely fresh"""
        try:
            for file in self.checkpoint_dir.glob("*"):
                file.unlink()
            logging.info("🧹 All checkpoint files cleared")
        except Exception as e:
            logging.error(f"❌ Failed to clear checkpoints: {e}")

    def process_dataset(self, df, output_path, resume=True):
        """Process entire dataset with batching and checkpointing"""
        start_time = datetime.now()
        
        # Load existing progress if resuming
        existing_results, processed_indices = (None, set())
        if resume:
            existing_results, processed_indices = self.load_checkpoint()
            
        # Filter out already processed locations
        remaining_df = df[~df.index.isin(processed_indices)].copy()
        
        if len(remaining_df) == 0:
            logging.info("🎉 All locations already processed!")
            if existing_results is not None:
                self._finalize_results(existing_results, df, output_path)
            return existing_results
            
        logging.info(f"🚀 Processing {len(remaining_df)} remaining locations in batches of {self.batch_size}")
        
        # Process in batches
        all_results = [existing_results] if existing_results is not None else []
        
        for batch_start in range(0, len(remaining_df), self.batch_size):
            batch_end = min(batch_start + self.batch_size, len(remaining_df))
            batch_df = remaining_df.iloc[batch_start:batch_end].copy()
            
            batch_num = len(processed_indices) // self.batch_size
            
            logging.info(f"🔄 Processing batch {batch_num + 1}: locations {batch_start + 1}-{batch_end}")
            
            # Process batch with progress tracking
            batch_results = []
            batch_start_time = datetime.now()
            
            for idx, row in tqdm(batch_df.iterrows(), 
                               total=len(batch_df),
                               desc=f"Batch {batch_num + 1}"):
                try:
                    result = compute_enhanced_Landsat_values(row)
                    result.name = idx
                    batch_results.append(result)
                except Exception as e:
                    logging.error(f"❌ Failed to process location {idx}: {e}")
                    # Create empty result
                    empty_result = pd.Series({col: np.nan for col in get_output_columns()})
                    empty_result.name = idx
                    batch_results.append(empty_result)
            
            # Convert to DataFrame
            batch_df_results = pd.DataFrame(batch_results)
            
            # Add metadata columns
            batch_df_results['Latitude'] = batch_df['Latitude'].values
            batch_df_results['Longitude'] = batch_df['Longitude'].values
            batch_df_results['Sample Date'] = batch_df['Sample Date'].values
            
            # Save checkpoint
            self.save_checkpoint(batch_num, batch_df_results, 
                               processed_indices | set(batch_df.index))
            
            all_results.append(batch_df_results)
            processed_indices.update(batch_df.index)
            
            # Calculate and log progress
            batch_time = (datetime.now() - batch_start_time).total_seconds()
            total_time = (datetime.now() - start_time).total_seconds()
            locations_per_second = len(batch_df) / batch_time
            
            remaining_locations = len(df) - len(processed_indices)
            eta_seconds = remaining_locations / locations_per_second if locations_per_second > 0 else 0
            eta_str = str(timedelta(seconds=int(eta_seconds)))
            
            logging.info(f"✅ Batch {batch_num + 1} completed in {batch_time:.1f}s")
            logging.info(f"📊 Progress: {len(processed_indices)}/{len(df)} locations ({len(processed_indices)/len(df)*100:.1f}%)")
            logging.info(f"⚡ Speed: {locations_per_second:.2f} locations/second")
            logging.info(f"⏰ ETA: {eta_str}")
            
        # Combine all results
        final_results = pd.concat(all_results, ignore_index=True)
        
        # Finalize and save
        self._finalize_results(final_results, df, output_path)
        
        total_time = (datetime.now() - start_time).total_seconds()
        logging.info(f"🎉 Processing completed in {total_time/3600:.2f} hours!")
        
        return final_results
        
    def _finalize_results(self, results_df, original_df, output_path):
        """Finalize results with proper column ordering"""
        
        # Ensure proper column ordering
        meta_cols = ['Latitude', 'Longitude', 'Sample Date']
        feature_cols = [col for col in get_output_columns() if col in results_df.columns]
        
        # Reorder columns
        final_columns = meta_cols + feature_cols
        results_df = results_df[final_columns]
        
        # Sort by original order if possible
        if len(results_df) == len(original_df):
            results_df = results_df.sort_index()
            
        # Save final results
        results_df.to_csv(output_path, index=False)
        logging.info(f"💾 Final results saved to {output_path}")
        
        # Create summary
        summary = {
            'total_locations': len(results_df),
            'successful_extractions': results_df['nir'].notna().sum(),
            'success_rate': f"{results_df['nir'].notna().sum() / len(results_df) * 100:.1f}%",
            'feature_count': len(feature_cols),
            'avg_cloud_cover': results_df['cloud_cover'].mean() if 'cloud_cover' in results_df else 'N/A'
        }
        
        logging.info("📊 Extraction Summary:")
        for key, value in summary.items():
            logging.info(f"   {key}: {value}")

print("🔧 Enhanced batch processor with robust error handling loaded!")

🔧 Enhanced batch processor with robust error handling loaded!


In [69]:
# 🚀 Enhanced Batch Processing - Choose Your Approach

# STEP 1: Choose your processing option
print("🎯 Choose your processing approach:")
print("   Uncomment ONE of the options below:")
print()

# Option A: Quick Test (recommended for first run)
#dataset_to_process = test_subset
#output_filename = "landsat_features_test.csv"
#batch_size = 5

# Option B: Medium Test (good for development)
#dataset_to_process = medium_subset
#output_filename = "landsat_features_medium.csv"
#batch_size = 50

# Option C: Full Production Run (uncomment for final extraction)
dataset_to_process = full_dataset
output_filename = "landsat_features_training.csv" 
batch_size = 200

print(f"✅ Selected: {len(dataset_to_process)} locations")
print(f"📁 Output file: {output_filename}")
print(f"📦 Batch size: {batch_size}")

# STEP 2: Initialize and run batch processor
processor = LandsatBatchProcessor(batch_size=batch_size)

print(f"\n🚀 Starting enhanced feature extraction...")
print(f"📊 Features to extract: {len(get_output_columns())}")
print(f"⏰ Estimated time: {len(dataset_to_process) * 2.5 / 60:.0f}-{len(dataset_to_process) * 4 / 60:.0f} minutes")

# Run the enhanced extraction
results = processor.process_dataset(
    df=dataset_to_process,
    output_path=output_filename,
    resume=True  # Set to False to start fresh
)

print(f"\n🎉 Extraction completed!")
print(f"📊 Results shape: {results.shape}")
print(f"💾 Saved to: {output_filename}")

# Show sample results
print(f"\n📋 Sample Results:")
display(results[['Latitude', 'Longitude', 'nir', 'green', 'NDVI', 'MNDWI', 'TurbidityIndex']].head())

2026-02-20 14:27:40,785 - INFO - 📂 Loaded checkpoint from batch_0001_checkpoint.json: 10 processed locations
2026-02-20 14:27:40,786 - INFO - 🚀 Processing 9309 remaining locations in batches of 200
2026-02-20 14:27:40,786 - INFO - 🔄 Processing batch 1: locations 1-200
2026-02-20 14:27:40,786 - INFO - 🚀 Processing 9309 remaining locations in batches of 200
2026-02-20 14:27:40,786 - INFO - 🔄 Processing batch 1: locations 1-200


🎯 Choose your processing approach:
   Uncomment ONE of the options below:

✅ Selected: 9319 locations
📁 Output file: landsat_features_training.csv
📦 Batch size: 200

🚀 Starting enhanced feature extraction...
📊 Features to extract: 28
⏰ Estimated time: 388-621 minutes


Batch 1: 100%|██████████| 200/200 [19:35<00:00,  5.88s/it]
2026-02-20 14:47:16,372 - INFO - ✅ Checkpoint saved for batch 0
2026-02-20 14:47:16,373 - INFO - ✅ Batch 1 completed in 1175.6s
2026-02-20 14:47:16,373 - INFO - 📊 Progress: 210/9319 locations (2.3%)
2026-02-20 14:47:16,373 - INFO - ⚡ Speed: 0.17 locations/second
2026-02-20 14:47:16,374 - INFO - ⏰ ETA: 14:52:22
2026-02-20 14:47:16,374 - INFO - 🔄 Processing batch 2: locations 201-400
Batch 2:   0%|          | 0/200 [00:00<?, ?it/s]
2026-02-20 14:47:16,372 - INFO - ✅ Checkpoint saved for batch 0
2026-02-20 14:47:16,373 - INFO - ✅ Batch 1 completed in 1175.6s
2026-02-20 14:47:16,373 - INFO - 📊 Progress: 210/9319 locations (2.3%)
2026-02-20 14:47:16,373 - INFO - ⚡ Speed: 0.17 locations/second
2026-02-20 14:47:16,374 - INFO - ⏰ ETA: 14:52:22
2026-02-20 14:47:16,374 - INFO - 🔄 Processing batch 2: locations 201-400
Batch 2: 100%|██████████| 200/200 [22:18<00:00,  6.69s/it]
2026-02-20 15:09:35,125 - INFO - ✅ Checkpoint saved for batch 1

KeyboardInterrupt: 

<h3>📊 Enhanced Feature Summary</h3>

<p>The enhanced extraction automatically computes <b>25+ features</b> compared to the baseline 6 features:</p>

<p><b>🛰️ Spectral Bands (7):</b></p>
<ul>
  <li><b>blue, green, red:</b> Visible spectrum bands for water color analysis</li>
  <li><b>nir:</b> Near-infrared for vegetation and water detection</li>
  <li><b>swir16, swir22:</b> Shortwave infrared for moisture analysis</li>
  <li><b>coastal:</b> Coastal aerosol band for atmospheric correction</li>
</ul>

<p><b>🌱 Vegetation Indices (6):</b></p>
<ul>
  <li><b>NDVI:</b> Normalized Difference Vegetation Index - vegetation health</li>
  <li><b>EVI:</b> Enhanced Vegetation Index - improved sensitivity</li>
  <li><b>SAVI:</b> Soil-Adjusted Vegetation Index - accounts for soil background</li>
  <li><b>ARVI:</b> Atmospherically Resistant Vegetation Index</li>
  <li><b>GNDVI:</b> Green Normalized Difference Vegetation Index</li>
  <li><b>RDVI:</b> Renormalized Difference Vegetation Index</li>
</ul>

<p><b>💧 Water Indices (4):</b></p>
<ul>
  <li><b>NDWI:</b> Normalized Difference Water Index - open water detection</li>
  <li><b>MNDWI:</b> Modified NDWI - enhanced water detection</li>
  <li><b>AWEInsh:</b> Automated Water Extraction Index (no shadows)</li>
  <li><b>AWEIsh:</b> Automated Water Extraction Index (with shadows)</li>
</ul>

<p><b>🏞️ Additional Indices (8+):</b></p>
<ul>
  <li><b>BSI, NDBI, UI:</b> Built-up and soil indices</li>
  <li><b>NBR:</b> Normalized Burn Ratio</li>
  <li><b>TurbidityIndex:</b> Water clarity assessment</li>
  <li><b>ChlorophyllIndex:</b> Algae and vegetation in water</li>
  <li><b>NDMI:</b> Normalized Difference Moisture Index</li>
  <li><b>Quality flags:</b> Cloud cover and data quality metrics</li>
</ul>

<p><b>🎯 Benefits for Water Quality Modeling:</b></p>
<ul>
  <li>Comprehensive spectral analysis instead of just 4 basic features</li>
  <li>Water-specific indices for turbidity and chlorophyll detection</li>
  <li>Quality flags for data reliability assessment</li>
  <li>Robust feature set that should significantly improve model performance</li>
</ul>


In [ ]:
# Results Analysis and Validation
if 'results' in locals() and results is not None:
    print("📊 Feature Extraction Analysis:")
    print(f"   Total locations processed: {len(results)}")
    
    # Success rate analysis
    success_rate = results['nir'].notna().sum() / len(results) * 100
    print(f"   Success rate: {success_rate:.1f}%")
    
    # Feature availability
    feature_cols = [col for col in results.columns if col not in ['Latitude', 'Longitude', 'Sample Date']]
    print(f"   Features extracted: {len(feature_cols)}")
    
    # Quality metrics
    if 'cloud_cover' in results.columns:
        avg_cloud = results['cloud_cover'].mean()
        print(f"   Average cloud cover: {avg_cloud:.1f}%")
    
    # Sample the results
    print(f"\n📋 Sample extracted features:")
    sample_cols = ['nir', 'green', 'NDVI', 'MNDWI', 'TurbidityIndex', 'ChlorophyllIndex']
    available_sample_cols = [col for col in sample_cols if col in results.columns]
    display(results[available_sample_cols].head())
    
else:
    print("⚠️  No results available. Please run the extraction above first.")

📊 Feature Extraction Analysis:
   Total locations processed: 10
   Success rate: 50.0%
   Features extracted: 28
   Average cloud cover: 0.8%

📋 Sample extracted features:


,nir,green,NDVI,MNDWI,TurbidityIndex,ChlorophyllIndex
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Finalization for Benchmark Notebook Integration
if 'results' in locals() and results is not None:
    
    # For benchmark compatibility, create the traditional variable name
    landsat_train_features = results.copy()
    
    print("✅ Results prepared for benchmark notebook integration")
    print(f"📊 Final dataset shape: {landsat_train_features.shape}")
    print(f"📁 Output file: {output_filename}")
    
    # Show column structure for verification
    print(f"\n📋 Column Structure:")
    meta_cols = ['Latitude', 'Longitude', 'Sample Date']
    feature_cols = [col for col in landsat_train_features.columns if col not in meta_cols]
    
    print(f"   Metadata columns ({len(meta_cols)}): {meta_cols}")
    print(f"   Feature columns ({len(feature_cols)}): {feature_cols[:10]}{'...' if len(feature_cols) > 10 else ''}")
    
else:
    print("⚠️  Please run the feature extraction first to generate results")

✅ Results prepared for benchmark notebook integration
📊 Final dataset shape: (10, 31)
📁 Output file: landsat_features_test.csv

📋 Column Structure:
   Metadata columns (3): ['Latitude', 'Longitude', 'Sample Date']
   Feature columns (28): ['blue', 'green', 'red', 'nir', 'swir16', 'swir22', 'coastal', 'NDVI', 'EVI', 'SAVI']...


In [ ]:
# Checkpoint: Save intermediate results (automatically handled by batch processor)
if 'landsat_train_features' in locals():
    print("💾 Results are automatically saved by the batch processor")
    print(f"📁 Latest file: {output_filename}")
    
    # Optional: Create a backup with timestamp
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_name = f"landsat_features_backup_{timestamp}.csv"
    landsat_train_features.to_csv(backup_name, index=False)
    print(f"🔄 Backup created: {backup_name}")
else:
    print("⚠️  No data to save. Please run the extraction first.")

💾 Results are automatically saved by the batch processor
📁 Latest file: landsat_features_test.csv
🔄 Backup created: landsat_features_backup_20260220_142521.csv


In [ ]:
# Preview Enhanced Results
if 'landsat_train_features' in locals():
    print("📊 Enhanced Landsat Features Preview:")
    display(landsat_train_features.head())
    
    # Feature summary
    print(f"\n📈 Feature Summary:")
    print(f"   Total samples: {len(landsat_train_features)}")
    print(f"   Total features: {len(landsat_train_features.columns) - 3}")  # Minus metadata
    print(f"   Success rate: {landsat_train_features['nir'].notna().sum() / len(landsat_train_features) * 100:.1f}%")
    
    # Show some key statistics
    if 'cloud_cover' in landsat_train_features.columns:
        print(f"   Avg cloud cover: {landsat_train_features['cloud_cover'].mean():.1f}%")
    
    print(f"\n🎯 Ready for water quality modeling with enhanced feature set!")
else:
    print("⚠️  No results to preview. Please run the extraction first.")

📊 Enhanced Landsat Features Preview:


,blue,green,red,nir,swir16,swir22,coastal,NDVI,EVI,SAVI,...,ClayMinerals,TurbidityIndex,ChlorophyllIndex,NIR_Red_Ratio,NDMI,cloud_cover,data_quality,Latitude,Longitude,Sample Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-28.760833,17.730278,02-01-2011
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-26.861111,28.884722,03-01-2011
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-26.450000,28.085833,03-01-2011
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-27.671111,27.236944,03-01-2011
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-27.356667,27.286389,03-01-2011



📈 Feature Summary:
   Total samples: 10
   Total features: 28
   Success rate: 50.0%
   Avg cloud cover: 0.8%

🎯 Ready for water quality modeling with enhanced feature set!


### 🎯 Enhanced Processing for Validation Dataset

Now let's apply the same enhanced feature extraction to the validation dataset using the same smart batching approach.

In [ ]:
# Load and analyze validation dataset
Validation_df = pd.read_csv('submission_template.csv')
print(f"📊 Loaded validation dataset: {Validation_df.shape}")
print(f"🎯 Total validation locations: {len(Validation_df)}")

display(Validation_df.head())

# Compare with training data
print(f"\n📈 Dataset Comparison:")
print(f"   Training locations: {len(Water_Quality_df) if 'Water_Quality_df' in locals() else 'N/A'}")
print(f"   Validation locations: {len(Validation_df)}")
print(f"   Validation columns: {list(Validation_df.columns)}")

📊 Loaded validation dataset: (200, 6)
🎯 Total validation locations: 200


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN



📈 Dataset Comparison:
   Training locations: 9319
   Validation locations: 200
   Validation columns: ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']


In [ ]:
Validation_df.shape

(200, 6)

In [ ]:
# Extract band values from Landsat for submission dataset
val_features_path = "landsat_features_validation.csv"

print("🚀 Running Landsat feature extraction for validation data...")
landsat_val_features = Validation_df.progress_apply(compute_Landsat_values, axis=1)
landsat_val_features.to_csv(val_features_path, index=False)

🚀 Running Landsat feature extraction for validation data...


NameError: name 'compute_Landsat_values' is not defined

In [ ]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_val_features['NDMI'] = (landsat_val_features['nir'] - landsat_val_features['swir16']) / (landsat_val_features['nir'] + landsat_val_features['swir16'])
landsat_val_features['MNDWI'] = (landsat_val_features['green'] - landsat_val_features['swir16']) / (landsat_val_features['green'] + landsat_val_features['swir16'] + eps)

In [ ]:
landsat_val_features['Latitude'] = Validation_df['Latitude']
landsat_val_features['Longitude'] = Validation_df['Longitude']
landsat_val_features['Sample Date'] = Validation_df['Sample Date']
landsat_val_features = landsat_val_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [ ]:
landsat_val_features.to_csv(val_features_path, index=False)

In [ ]:
# Preview File
landsat_val_features.head()